In [23]:
import os
import sys
import random
import pandas
import numpy
import re

In [38]:
HOME_DIR='./'
PERT_FRAC = 0.0
OUT_FOLDER = str(PERT_FRAC).replace('.', '_')
ROOT_FOLDER = 'rps'

In [39]:
def original_leak_sections():
    sections = [
        [],
        [],
        [
            'LeakWasteAC = sA mA fA hAk( + sC* ) fA*( hAk*( fA*( + ) ) ) @initial 0 nM',
             'LeakWasteBA = sB mB fB hBr( + sA* ) fB*( hBr*( fB*( + ) ) ) @initial 0 nM',
             'LeakWasteCB = sC mC fC hCj( + sB* ) fC*( hCj*( fC*( + ) ) ) @initial 0 nM'
        ],
        [
            'macrostate LeakWasteCB = [LeakWasteCB]',
             'macrostate LeakWasteBA = [LeakWasteBA]',
             'macrostate LeakWasteAC = [LeakWasteAC]'
        ],
        [
            'reaction [condensed    = 2e-7 /nM/s ] ProduceBCjCj + HelperCCj -> Cj + LeakWasteCB',
             'reaction [condensed    = 2e-7 /nM/s ] ProduceABrBr + HelperBBr -> Br + LeakWasteBA',
             'reaction [condensed    = 2e-7 /nM/s ] ProduceCAkAk + HelperAAk -> Ak + LeakWasteAC'
        ]
    ]
    return sections

def shadow_leak_sections():
    sections = [
        [],
        [],
        [
            'sLeakWasteAC = hAkR( fAR mAR sAR + fAR( hAkR( fAR( + ) ) ) ) sCR* @initial 0 nM',
             'sLeakWasteBA = hBrR( fBR mBR sBR + fBR( hBrR( fBR( + ) ) ) ) sAR* @initial 0 nM',
             'sLeakWasteCB = hCjR( fCR mCR sCR + fCR( hCjR( fCR( + ) ) ) ) sBR* @initial 0 nM'
        ],
        [
            'macrostate sLeakWasteAC = [sLeakWasteAC]',
             'macrostate sLeakWasteBA = [sLeakWasteBA]',
             'macrostate sLeakWasteCB = [sLeakWasteCB]'
        ],
        [
            'reaction [condensed    = 2e-7 /nM/s ] sProduceBCjCj + sHelperCCj -> sCj + sLeakWasteCB',
             'reaction [condensed   = 2e-7 /nM/s ] sProduceABrBr + sHelperBBr -> sBr + sLeakWasteBA',
             'reaction [condensed   = 2e-7 /nM/s ] sProduceCAkAk + sHelperAAk -> sAk + sLeakWasteAC'
        ]
    ]
    return sections

In [40]:
def read_lines(file):
    with open(file, 'r') as fin:
        lines = fin.readlines()
        lines = [line.strip() for line in lines]
    return lines

def write_lines(lines, file):
    with open(file, 'w') as fout:
        for line in lines:
            fout.write(line)
            fout.write('\n')
            
def eps_ub(d, frac):
    l = [d + d*frac]
    return round(random.choice(l), 9)

def match_replace_rate_const(s, regex='([0-9]\.[0-9]+)', frac=None):
    x = re.search(regex, s)
    if x is not None:
        rep = str(eps_ub(float(x.group()), frac))
        ret = re.sub(regex, rep, s)
        return ret
    else:
        return s

def divide_sections(lines):
    line_numbers = []
    for i, line in enumerate(lines):
        if '#' in line:
            line_numbers.append(i)
    line_numbers.append(len(lines))
    sections = []
    headers = []
    for i in range(len(line_numbers)-1):
        start = line_numbers[i]
        end = line_numbers[i+1]
        sections.append(lines[start+1:end])
        headers.append(lines[start])
    return headers, sections


def merge(ofile, sfile, cfile=None):
    olines = read_lines(ofile)
    slines = read_lines(sfile)
    
    o_heds, o_secs = divide_sections(olines)
    s_heds, s_secs = divide_sections(slines)
    
    o_leak_secs = original_leak_sections() 
    s_leak_secs = shadow_leak_sections()
    
    assert len(o_secs) == len(s_secs)
    assert len(o_secs) == len(o_leak_secs)
    
    headers = o_heds
    sections = [x[0] + x[1] + x[2] + x[3] for x in zip(o_secs, s_secs, o_leak_secs, s_leak_secs)]
   
    
    if cfile is not None:
        clines = read_lines(cfile)
        c_heds, c_secs = divide_sections(clines)
        assert len(c_secs) == len(o_secs)
        sections = [x[0] + list(set(x[1]).difference(set(x[0]))) for x in zip(sections, c_secs)]
    
    return headers, sections

def write_out(headers, sections, out=None, cancel=False):
    lines = []
    for h, s in zip(headers, sections):
        lines += [h]
        lines += s
    subbed_lines = []
    if cancel:
        for line in lines:
            if re.match(r'sCj = hCjR fCR mCR sCR @initial [0-9]+ nM', line) is not None:
                subbed_lines += ['sCj = hCjR fCR mCR sCR @initial 0.5 nM']
            elif re.match(r'sAk = hAkR fAR mAR sAR @initial [0-9]+ nM', line) is not None:
                subbed_lines += ['sAk = hAkR fAR mAR sAR @initial 0.3 nM']
            elif re.match(r'sBr = hBrR fBR mBR sBR @initial [0-9]+ nM', line) is not None:
                subbed_lines += ['sBr = hBrR fBR mBR sBR @initial 0.2 nM']
            else:
                subbed_lines.append(line)
                                
        write_lines(subbed_lines, out)
    else:
        write_lines(lines, out)

# Add leaks to Original, Shadow, and Merge UnPerturbed

In [41]:
OUT_DIR = os.path.join(HOME_DIR, ROOT_FOLDER, OUT_FOLDER)
if not os.path.exists(OUT_DIR):
    os.mkdir(OUT_DIR)


In [42]:
ofile = os.path.join(HOME_DIR, ROOT_FOLDER, 'vanilla', 'main_enum.pil')
sfile = os.path.join(HOME_DIR, ROOT_FOLDER, 'shadow_vanilla', 'main_enum.pil')
cfile = os.path.join(HOME_DIR, ROOT_FOLDER, 'cancel', 'main_enum.pil')
out_nocancel = os.path.join(OUT_DIR, 'orig_shadow_nocancel_enum.pil')
out_cancel = os.path.join(OUT_DIR, 'orig_shadow_cancel_enum.pil')

headers, sections = merge(ofile, sfile)
write_out(headers, sections, out_nocancel)
headers, sections = merge(ofile, sfile, cfile)
write_out(headers, sections, out_cancel, cancel=True)

# Perturb

In [43]:
lines = read_lines(os.path.join(HOME_DIR, ROOT_FOLDER, 'shadow_vanilla', 'main_enum.pil'))
pert_lines = []
for line in lines:
    pert_lines.append(match_replace_rate_const(line, frac=PERT_FRAC))
write_lines(pert_lines, os.path.join(HOME_DIR, ROOT_FOLDER, 'shadow_vanilla', f'main_pert-{PERT_FRAC}_enum.pil'))

# Add leaks to Original, Shadow, and Perturbed

In [44]:
ofile = os.path.join(HOME_DIR, ROOT_FOLDER, 'vanilla', 'main_enum.pil')
sfile = os.path.join(HOME_DIR, ROOT_FOLDER, 'shadow_vanilla', f'main_pert-{PERT_FRAC}_enum.pil')
cfile = os.path.join(HOME_DIR, ROOT_FOLDER, 'cancel', 'main_enum.pil')
out_nocancel = os.path.join(OUT_DIR, 'orig_shadow_pert_nocancel_enum.pil')
out_cancel = os.path.join(OUT_DIR, 'orig_shadow_pert_cancel_enum.pil')

headers, sections = merge(ofile, sfile)
write_out(headers, sections, out_nocancel)
headers, sections = merge(ofile, sfile, cfile)
write_out(headers, sections, out_cancel, cancel=True)